In [11]:
# Created By: Farhan Choudhary
# Talk to my Chatbot here: https://cdn.rawgit.com/farhanchoudhary/Chatbot/cc27043b/chatbot.html
# Connect on LinkedIn: www.linkedin.com/in/farhanchoudhary

# Deep Convolutional GANs

# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [12]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

In [13]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.


In [14]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.


Files already downloaded and verified


In [15]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [16]:
# Defining the generator

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [17]:
# Creating the generator
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [18]:
# Defining the discriminator

class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)


In [19]:
# Creating the discriminator
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [20]:
# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))


In [23]:
for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item()))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/25][0/782] Loss_D: 1.0323 Loss_G: 6.5051
[0/25][1/782] Loss_D: 0.7856 Loss_G: 5.2404
[0/25][2/782] Loss_D: 1.1510 Loss_G: 6.0433
[0/25][3/782] Loss_D: 1.0809 Loss_G: 6.6950
[0/25][4/782] Loss_D: 1.0883 Loss_G: 6.7366
[0/25][5/782] Loss_D: 0.7687 Loss_G: 7.6318
[0/25][6/782] Loss_D: 0.9062 Loss_G: 8.9415
[0/25][7/782] Loss_D: 0.6376 Loss_G: 7.4918
[0/25][8/782] Loss_D: 1.0263 Loss_G: 8.7135
[0/25][9/782] Loss_D: 0.5938 Loss_G: 8.3140
[0/25][10/782] Loss_D: 0.5356 Loss_G: 9.1954
[0/25][11/782] Loss_D: 0.4085 Loss_G: 9.3362
[0/25][12/782] Loss_D: 0.4590 Loss_G: 10.1788
[0/25][13/782] Loss_D: 0.7234 Loss_G: 10.4871
[0/25][14/782] Loss_D: 0.4414 Loss_G: 11.5649
[0/25][15/782] Loss_D: 0.4307 Loss_G: 10.0620
[0/25][16/782] Loss_D: 0.6294 Loss_G: 12.8987
[0/25][17/782] Loss_D: 0.3711 Loss_G: 9.1159
[0/25][18/782] Loss_D: 0.8082 Loss_G: 16.4514
[0/25][19/782] Loss_D: 0.2271 Loss_G: 14.7997
[0/25][20/782] Loss_D: 0.1196 Loss_G: 8.0910
[0/25][21/782] Loss_D: 2.5478 Loss_G: 20.5728
[0/25][22/78

[0/25][178/782] Loss_D: 0.0005 Loss_G: 38.6253
[0/25][179/782] Loss_D: 0.0020 Loss_G: 38.6077
[0/25][180/782] Loss_D: 0.0015 Loss_G: 38.4287
[0/25][181/782] Loss_D: 0.0008 Loss_G: 38.6464
[0/25][182/782] Loss_D: 0.0003 Loss_G: 38.3173
[0/25][183/782] Loss_D: 0.0005 Loss_G: 38.4099
[0/25][184/782] Loss_D: 0.0004 Loss_G: 38.4567
[0/25][185/782] Loss_D: 0.0007 Loss_G: 38.4049
[0/25][186/782] Loss_D: 0.0006 Loss_G: 38.5103
[0/25][187/782] Loss_D: 0.0008 Loss_G: 38.4934
[0/25][188/782] Loss_D: 0.0007 Loss_G: 38.4825
[0/25][189/782] Loss_D: 0.0004 Loss_G: 38.4015
[0/25][190/782] Loss_D: 0.0021 Loss_G: 38.5323
[0/25][191/782] Loss_D: 0.0003 Loss_G: 38.4160
[0/25][192/782] Loss_D: 0.0002 Loss_G: 38.7340
[0/25][193/782] Loss_D: 0.0010 Loss_G: 38.5392
[0/25][194/782] Loss_D: 0.0013 Loss_G: 38.3728
[0/25][195/782] Loss_D: 0.0008 Loss_G: 38.5246
[0/25][196/782] Loss_D: 0.0008 Loss_G: 38.5521
[0/25][197/782] Loss_D: 0.0009 Loss_G: 38.5962
[0/25][198/782] Loss_D: 0.0007 Loss_G: 38.5003
[0/25][199/78

[0/25][353/782] Loss_D: 0.0001 Loss_G: 34.5794
[0/25][354/782] Loss_D: 0.0007 Loss_G: 34.1975
[0/25][355/782] Loss_D: 0.0002 Loss_G: 33.4351
[0/25][356/782] Loss_D: 0.0007 Loss_G: 32.5622
[0/25][357/782] Loss_D: 0.0002 Loss_G: 31.2875
[0/25][358/782] Loss_D: 0.0001 Loss_G: 28.7882
[0/25][359/782] Loss_D: 0.0001 Loss_G: 23.2391
[0/25][360/782] Loss_D: 0.0009 Loss_G: 9.8032
[0/25][361/782] Loss_D: 6.1620 Loss_G: 37.4320
[0/25][362/782] Loss_D: 8.7693 Loss_G: 37.5647
[0/25][363/782] Loss_D: 4.4494 Loss_G: 33.6804
[0/25][364/782] Loss_D: 0.4896 Loss_G: 30.7863
[0/25][365/782] Loss_D: 0.0331 Loss_G: 27.8526
[0/25][366/782] Loss_D: 0.0042 Loss_G: 23.7903
[0/25][367/782] Loss_D: 0.0098 Loss_G: 14.8941
[0/25][368/782] Loss_D: 0.6555 Loss_G: 14.6452
[0/25][369/782] Loss_D: 0.0157 Loss_G: 12.7206
[0/25][370/782] Loss_D: 0.1924 Loss_G: 6.8881
[0/25][371/782] Loss_D: 2.0461 Loss_G: 24.9619
[0/25][372/782] Loss_D: 2.3290 Loss_G: 25.7514
[0/25][373/782] Loss_D: 2.4205 Loss_G: 21.7923
[0/25][374/782]

[0/25][531/782] Loss_D: 2.2743 Loss_G: 12.6371
[0/25][532/782] Loss_D: 0.8298 Loss_G: 5.9368
[0/25][533/782] Loss_D: 0.6352 Loss_G: 6.2708
[0/25][534/782] Loss_D: 0.2234 Loss_G: 6.9616
[0/25][535/782] Loss_D: 0.3301 Loss_G: 5.0465
[0/25][536/782] Loss_D: 0.4049 Loss_G: 7.1467
[0/25][537/782] Loss_D: 0.1672 Loss_G: 6.0444
[0/25][538/782] Loss_D: 0.1779 Loss_G: 5.9884
[0/25][539/782] Loss_D: 0.4530 Loss_G: 9.4630
[0/25][540/782] Loss_D: 0.7655 Loss_G: 5.0481
[0/25][541/782] Loss_D: 0.9404 Loss_G: 11.5759
[0/25][542/782] Loss_D: 0.5240 Loss_G: 9.6202
[0/25][543/782] Loss_D: 0.3259 Loss_G: 5.0541
[0/25][544/782] Loss_D: 0.6921 Loss_G: 11.7064
[0/25][545/782] Loss_D: 1.3726 Loss_G: 7.0192
[0/25][546/782] Loss_D: 0.1788 Loss_G: 4.0364
[0/25][547/782] Loss_D: 0.6819 Loss_G: 11.0380
[0/25][548/782] Loss_D: 0.4991 Loss_G: 9.9517
[0/25][549/782] Loss_D: 0.2411 Loss_G: 6.9385
[0/25][550/782] Loss_D: 0.1664 Loss_G: 6.1432
[0/25][551/782] Loss_D: 0.2725 Loss_G: 9.5813
[0/25][552/782] Loss_D: 0.5138

[0/25][709/782] Loss_D: 0.2419 Loss_G: 3.4891
[0/25][710/782] Loss_D: 0.2862 Loss_G: 4.8273
[0/25][711/782] Loss_D: 0.1979 Loss_G: 4.5228
[0/25][712/782] Loss_D: 0.2610 Loss_G: 3.2866
[0/25][713/782] Loss_D: 0.3151 Loss_G: 4.3184
[0/25][714/782] Loss_D: 0.3155 Loss_G: 4.0543
[0/25][715/782] Loss_D: 0.3164 Loss_G: 3.5589
[0/25][716/782] Loss_D: 0.4529 Loss_G: 2.2570
[0/25][717/782] Loss_D: 0.7785 Loss_G: 6.3368
[0/25][718/782] Loss_D: 0.9430 Loss_G: 1.6837
[0/25][719/782] Loss_D: 1.7548 Loss_G: 9.6838
[0/25][720/782] Loss_D: 1.8054 Loss_G: 1.2735
[0/25][721/782] Loss_D: 1.6487 Loss_G: 6.9604
[0/25][722/782] Loss_D: 1.5807 Loss_G: 2.7183
[0/25][723/782] Loss_D: 1.3343 Loss_G: 3.5701
[0/25][724/782] Loss_D: 1.4265 Loss_G: 3.1664
[0/25][725/782] Loss_D: 1.2197 Loss_G: 3.3087
[0/25][726/782] Loss_D: 0.8790 Loss_G: 2.7856
[0/25][727/782] Loss_D: 0.8157 Loss_G: 3.3082
[0/25][728/782] Loss_D: 0.5473 Loss_G: 3.7612
[0/25][729/782] Loss_D: 0.5305 Loss_G: 3.3482
[0/25][730/782] Loss_D: 0.4993 Los

...

[23/25][755/782] Loss_D: 0.0886 Loss_G: 4.0761
[23/25][756/782] Loss_D: 0.1216 Loss_G: 4.4624
[23/25][757/782] Loss_D: 0.1713 Loss_G: 3.7914
[23/25][758/782] Loss_D: 0.2305 Loss_G: 3.6625
[23/25][759/782] Loss_D: 0.1372 Loss_G: 4.0950
[23/25][760/782] Loss_D: 0.1255 Loss_G: 4.0364
[23/25][761/782] Loss_D: 0.1180 Loss_G: 3.8829
[23/25][762/782] Loss_D: 0.1475 Loss_G: 3.4786
[23/25][763/782] Loss_D: 0.1255 Loss_G: 4.1922
[23/25][764/782] Loss_D: 0.1506 Loss_G: 4.3653
[23/25][765/782] Loss_D: 0.1824 Loss_G: 3.7655
[23/25][766/782] Loss_D: 0.2100 Loss_G: 2.9106
[23/25][767/782] Loss_D: 0.2269 Loss_G: 4.7159
[23/25][768/782] Loss_D: 0.1798 Loss_G: 4.6297
[23/25][769/782] Loss_D: 0.7835 Loss_G: 0.8867
[23/25][770/782] Loss_D: 1.1833 Loss_G: 7.4725
[23/25][771/782] Loss_D: 0.3323 Loss_G: 4.7142
[23/25][772/782] Loss_D: 0.1844 Loss_G: 2.9253
[23/25][773/782] Loss_D: 0.2611 Loss_G: 4.4915
[23/25][774/782] Loss_D: 0.2044 Loss_G: 4.9601
[23/25][775/782] Loss_D: 0.1454 Loss_G: 4.3583
[23/25][776/7

[24/25][150/782] Loss_D: 1.0902 Loss_G: 5.6747
[24/25][151/782] Loss_D: 1.0865 Loss_G: 1.5552
[24/25][152/782] Loss_D: 1.6267 Loss_G: 6.2090
[24/25][153/782] Loss_D: 1.0138 Loss_G: 2.7609
[24/25][154/782] Loss_D: 0.5262 Loss_G: 1.3195
[24/25][155/782] Loss_D: 1.4694 Loss_G: 6.7021
[24/25][156/782] Loss_D: 1.3327 Loss_G: 2.1597
[24/25][157/782] Loss_D: 0.6851 Loss_G: 2.7815
[24/25][158/782] Loss_D: 1.0876 Loss_G: 1.3237
[24/25][159/782] Loss_D: 1.7146 Loss_G: 6.7332
[24/25][160/782] Loss_D: 1.5969 Loss_G: 2.2253
[24/25][161/782] Loss_D: 1.1832 Loss_G: 2.2727
[24/25][162/782] Loss_D: 1.1319 Loss_G: 5.3312
[24/25][163/782] Loss_D: 0.5378 Loss_G: 3.5020
[24/25][164/782] Loss_D: 0.4392 Loss_G: 4.2434
[24/25][165/782] Loss_D: 0.2201 Loss_G: 4.5974
[24/25][166/782] Loss_D: 0.2833 Loss_G: 3.5567
[24/25][167/782] Loss_D: 0.4209 Loss_G: 5.6538
[24/25][168/782] Loss_D: 0.8099 Loss_G: 1.1638
[24/25][169/782] Loss_D: 1.1340 Loss_G: 7.7389
[24/25][170/782] Loss_D: 0.8115 Loss_G: 4.0516
[24/25][171/7

[24/25][325/782] Loss_D: 0.0442 Loss_G: 5.2437
[24/25][326/782] Loss_D: 0.1053 Loss_G: 4.6692
[24/25][327/782] Loss_D: 0.0681 Loss_G: 4.0197
[24/25][328/782] Loss_D: 0.1524 Loss_G: 5.4096
[24/25][329/782] Loss_D: 0.3170 Loss_G: 3.9951
[24/25][330/782] Loss_D: 0.2611 Loss_G: 3.7619
[24/25][331/782] Loss_D: 0.4311 Loss_G: 0.4956
[24/25][332/782] Loss_D: 1.5826 Loss_G: 15.4028
[24/25][333/782] Loss_D: 6.8140 Loss_G: 1.9838
[24/25][334/782] Loss_D: 0.6366 Loss_G: 3.4114
[24/25][335/782] Loss_D: 0.5303 Loss_G: 3.0737
[24/25][336/782] Loss_D: 0.3695 Loss_G: 2.8766
[24/25][337/782] Loss_D: 0.3661 Loss_G: 4.8574
[24/25][338/782] Loss_D: 0.2974 Loss_G: 3.4631
[24/25][339/782] Loss_D: 0.3457 Loss_G: 4.6592
[24/25][340/782] Loss_D: 0.2223 Loss_G: 3.9312
[24/25][341/782] Loss_D: 0.2830 Loss_G: 2.9201
[24/25][342/782] Loss_D: 0.3161 Loss_G: 5.7141
[24/25][343/782] Loss_D: 0.2824 Loss_G: 3.8781
[24/25][344/782] Loss_D: 0.1629 Loss_G: 3.9392
[24/25][345/782] Loss_D: 0.0426 Loss_G: 5.4433
[24/25][346/

[24/25][500/782] Loss_D: 0.2363 Loss_G: 3.8353
[24/25][501/782] Loss_D: 0.2214 Loss_G: 3.8179
[24/25][502/782] Loss_D: 0.1572 Loss_G: 3.7509
[24/25][503/782] Loss_D: 0.1188 Loss_G: 4.0670
[24/25][504/782] Loss_D: 0.1353 Loss_G: 4.1288
[24/25][505/782] Loss_D: 0.1008 Loss_G: 4.4171
[24/25][506/782] Loss_D: 0.1129 Loss_G: 3.9292
[24/25][507/782] Loss_D: 0.1005 Loss_G: 3.9685
[24/25][508/782] Loss_D: 0.1535 Loss_G: 3.7171
[24/25][509/782] Loss_D: 0.1198 Loss_G: 3.9113
[24/25][510/782] Loss_D: 0.2334 Loss_G: 3.2091
[24/25][511/782] Loss_D: 0.2192 Loss_G: 3.6297
[24/25][512/782] Loss_D: 0.1723 Loss_G: 3.8149
[24/25][513/782] Loss_D: 0.2722 Loss_G: 2.8519
[24/25][514/782] Loss_D: 0.1483 Loss_G: 3.6587
[24/25][515/782] Loss_D: 0.2049 Loss_G: 2.9600
[24/25][516/782] Loss_D: 0.3216 Loss_G: 5.7110
[24/25][517/782] Loss_D: 0.2996 Loss_G: 4.2086
[24/25][518/782] Loss_D: 0.2713 Loss_G: 1.8077
[24/25][519/782] Loss_D: 0.3813 Loss_G: 5.5003
[24/25][520/782] Loss_D: 0.1281 Loss_G: 5.0648
[24/25][521/7

[24/25][675/782] Loss_D: 0.6600 Loss_G: 1.8240
[24/25][676/782] Loss_D: 0.6342 Loss_G: 5.0692
[24/25][677/782] Loss_D: 0.3496 Loss_G: 3.1346
[24/25][678/782] Loss_D: 0.1914 Loss_G: 3.0349
[24/25][679/782] Loss_D: 0.4434 Loss_G: 4.9531
[24/25][680/782] Loss_D: 0.6447 Loss_G: 2.1205
[24/25][681/782] Loss_D: 0.8042 Loss_G: 5.6490
[24/25][682/782] Loss_D: 0.6371 Loss_G: 2.6624
[24/25][683/782] Loss_D: 0.2859 Loss_G: 3.3041
[24/25][684/782] Loss_D: 0.2112 Loss_G: 4.8177
[24/25][685/782] Loss_D: 0.1463 Loss_G: 4.7236
[24/25][686/782] Loss_D: 0.3014 Loss_G: 2.4009
[24/25][687/782] Loss_D: 0.3089 Loss_G: 4.1129
[24/25][688/782] Loss_D: 0.2996 Loss_G: 2.8176
[24/25][689/782] Loss_D: 0.4747 Loss_G: 5.6544
[24/25][690/782] Loss_D: 0.1754 Loss_G: 5.0330
[24/25][691/782] Loss_D: 0.3296 Loss_G: 2.0945
[24/25][692/782] Loss_D: 0.4888 Loss_G: 4.1282
[24/25][693/782] Loss_D: 0.1526 Loss_G: 4.0784
[24/25][694/782] Loss_D: 0.1190 Loss_G: 3.9839
[24/25][695/782] Loss_D: 0.1431 Loss_G: 3.8719
[24/25][696/7